# SageMaker on Local device
1. able to request on local device
2. able to run on local device
    - use `instance_type='local'`
3. able to deploy endpoint on local device


[usage](https://www.youtube.com/watch?v=K3ngZKF31mc)
1. pip install sagemaker
2. aws iam list-roles | grep SageMaker-Execution
3. set iam role

[sample](https://github.com/aws-samples/amazon-sagemaker-local-mode)
- set the parameters below for processor or estimator
    - set the session `sagemaker_session=sess`
    - set the role `role=role`
    - set the instance type `instance_type='local'` if want to run on local device
- data should be in s3 if it's not local mode


In [2]:
from util.sagemaker_util import *

# change to the iam role on `aws iam list-roles | grep SageMaker-Execution`
local_role = "arn:aws:iam::111111111111:role/service-role/AmazonSageMaker-ExecutionRole-20200101T000001"

set_environment(
    local_role=local_role,
    is_local=True,
    is_debug=True,
    region_name='us-east-1',
    profile_name='default',
    zip_s3_path="s3://bucket/path/to/model.zip",
)
upload_pyfiles()

1.21.3
2.76.0
arn:aws:iam::111111111111:role/service-role/AmazonSageMaker-ExecutionRole-20200101T000001
sagemaker-ap-south-1-206034181024


In [9]:
# This is a sample Python program that trains a simple XGBoost model on Abalone dataset.
# This implementation will work on your *local computer* or in the *AWS Cloud*.
#
# Prerequisites:
#   1. Install required Python packages:
#      `pip install -r requirements.txt`
#   2. Docker Desktop installed and running on your computer:
#      `docker ps`
#   3. You should have AWS credentials configured on your local machine
#      in order to be able to pull the docker image from ECR.
###############################################################################################

from sagemaker import TrainingInput
from sagemaker.xgboost import XGBoost, XGBoostModel

def do_inference_on_local_endpoint(predictor, libsvm_str):
    label, *features = libsvm_str.strip().split()
    predictions = predictor.predict(" ".join(["-99"] + features))  # use dummy label -99
    print("Prediction: {}".format(predictions))


def main():
    print('Starting model training.')
    print('Note: if launching for the first time in local mode, container image download might take a few minutes to complete.')

    hyperparameters = {
        "max_depth": "5",
        "eta": "0.2",
        "gamma": "4",
        "min_child_weight": "6",
        "subsample": "0.7",
        "objective": "reg:squarederror",
        "num_round": "50",
        "verbosity": "2",
    }

    xgb_script_mode_estimator = XGBoost(
        entry_point="./code/abalone.py",
        hyperparameters=hyperparameters,
        instance_count=1,
        framework_version="1.2-1",
        **param('ml.m5.2xlarge')
    )

    train_input = TrainingInput("s3://some_bucket/abalone", content_type="text/libsvm")

    xgb_script_mode_estimator.fit({"train": train_input, "validation": train_input})

    print('Completed model training')

    model_data = xgb_script_mode_estimator.model_data
    print(model_data)

    xgb_inference_model = XGBoostModel(
        model_data=model_data,
        entry_point="./code/inference.py",
        framework_version="1.2-1",
        **param()
    )

    print('Deploying endpoint in local mode')
    predictor = xgb_inference_model.deploy(
        initial_instance_count=1,
        instance_type=instance_type('ml.m5.2xlarge'),
    )

    a_young_abalone = "6 1:3 2:0.37 3:0.29 4:0.095 5:0.249 6:0.1045 7:0.058 8:0.067"
    do_inference_on_local_endpoint(predictor, a_young_abalone)

    an_old_abalone = "15 1:1 2:0.655 3:0.53 4:0.175 5:1.2635 6:0.486 7:0.2635 8:0.415"
    do_inference_on_local_endpoint(predictor, an_old_abalone)

    if not is_local():
        print('About to delete the endpoint to stop paying (if in cloud mode).')
        predictor.delete_endpoint(predictor.endpoint_name)


main()


Starting model training.
Note: if launching for the first time in local mode, container image download might take a few minutes to complete.
2022-02-20 14:12:42 Starting - Starting the training job...
2022-02-20 14:13:07 Starting - Preparing the instances for trainingProfilerReport-1645366361: InProgress
......
2022-02-20 14:14:09 Downloading - Downloading input data
2022-02-20 14:14:09 Training - Downloading the training image......
2022-02-20 14:15:28 Uploading - Uploading generated training model[2022-02-20 14:15:12.734 ip-10-0-126-143.ap-south-1.compute.internal:1 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Invoking user training script.
INFO:sagemaker-containers:Module abalone does not provide a setup.py. 
Generating setup.py
INFO:sagemaker-containers:Generating setup.cfg
INFO: